 Licensed Materials - Property of IBM (c) Copyright IBM Corp. 2023 All Rights Reserved.

 US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with
 IBM Corp.

 DISCLAIMER OF WARRANTIES :

 Permission is granted to copy and modify this Sample code, and to distribute modified versions provided that both the
 copyright notice, and this permission notice and warranty disclaimer appear in all copies and modified versions.

 THIS SAMPLE CODE IS LICENSED TO YOU AS-IS. IBM AND ITS SUPPLIERS AND LICENSORS DISCLAIM ALL WARRANTIES, EITHER
 EXPRESS OR IMPLIED, IN SUCH SAMPLE CODE, INCLUDING THE WARRANTY OF NON-INFRINGEMENT AND THE IMPLIED WARRANTIES OF
 MERCHANTABILITY OR FITNESS FOR A PARTICULAR PURPOSE. IN NO EVENT WILL IBM OR ITS LICENSORS OR SUPPLIERS BE LIABLE FOR
 ANY DAMAGES ARISING OUT OF THE USE OF OR INABILITY TO USE THE SAMPLE CODE, DISTRIBUTION OF THE SAMPLE CODE, OR
 COMBINATION OF THE SAMPLE CODE WITH ANY OTHER CODE. IN NO EVENT SHALL IBM OR ITS LICENSORS AND SUPPLIERS BE LIABLE
 FOR ANY LOST REVENUE, LOST PROFITS OR DATA, OR FOR DIRECT, INDIRECT, SPECIAL, CONSEQUENTIAL, INCIDENTAL OR PUNITIVE
 DAMAGES, HOWEVER CAUSED AND REGARDLESS OF THE THEORY OF LIABILITY, EVEN IF IBM OR ITS LICENSORS OR SUPPLIERS HAVE
 BEEN ADVISED OF THE POSSIBILITY OF SUCH DAMAGES.

 <!-- Licensed Materials - Property of IBM (c) Copyright IBM Corp. 2023 All Rights Reserved.

 US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with
 IBM Corp.

 DISCLAIMER OF WARRANTIES :

 Permission is granted to copy and modify this Sample code, and to distribute modified versions provided that both the
 copyright notice, and this permission notice and warranty disclaimer appear in all copies and modified versions.

 THIS SAMPLE CODE IS LICENSED TO YOU AS-IS. IBM AND ITS SUPPLIERS AND LICENSORS DISCLAIM ALL WARRANTIES, EITHER
 EXPRESS OR IMPLIED, IN SUCH SAMPLE CODE, INCLUDING THE WARRANTY OF NON-INFRINGEMENT AND THE IMPLIED WARRANTIES OF
 MERCHANTABILITY OR FITNESS FOR A PARTICULAR PURPOSE. IN NO EVENT WILL IBM OR ITS LICENSORS OR SUPPLIERS BE LIABLE FOR
 ANY DAMAGES ARISING OUT OF THE USE OF OR INABILITY TO USE THE SAMPLE CODE, DISTRIBUTION OF THE SAMPLE CODE, OR
 COMBINATION OF THE SAMPLE CODE WITH ANY OTHER CODE. IN NO EVENT SHALL IBM OR ITS LICENSORS AND SUPPLIERS BE LIABLE
 FOR ANY LOST REVENUE, LOST PROFITS OR DATA, OR FOR DIRECT, INDIRECT, SPECIAL, CONSEQUENTIAL, INCIDENTAL OR PUNITIVE
 DAMAGES, HOWEVER CAUSED AND REGARDLESS OF THE THEORY OF LIABILITY, EVEN IF IBM OR ITS LICENSORS OR SUPPLIERS HAVE
 BEEN ADVISED OF THE POSSIBILITY OF SUCH DAMAGES. -->

In [ ]:
import json
import config
from demoutil import ClassDefinitionsSelector
from csdeploy.gqlinvoke import GraphqlConnection
from csdeploy.gqlinvoke import GraphqlRequest
from csdeploy.impexp import Exporter
from csdeploy.pkg import DeploymentPackage
from csdeploy.pkg import ExportSelection, ClassDefinitionSelection
from csdeploy.impexp import Importer


# The systems for export and import

In [ ]:
print("Export:", config.GQL_URL, config.OS_NAME)
print("\n")
print("Import:", config.TARGET_GQL_URL, config.TARGET_OS_NAME)

# Authenticate with the source GraphQL server
(Choose one method of Authentication)


#### OAuth Authentication

In [ ]:
source_connection = GraphqlConnection(url=config.GQL_URL, token_url=config.TOKEN_URL)
source_connection.initialize_oauth(client_id=config.CLIENT_ID,client_secret= config.CLIENT_SECRET, username=config.OAUTH_USERNAME, password=config.OAUTH_PASSWORD,\
                                grant_type=config.GRANT_TYPE, scope=config.SCOPE)
source_connection.get_token()
print("XSRF token: " + source_connection.xsrf_token)
print("Authentication token: " + source_connection.token)

#### Basic Authentication

In [ ]:
source_connection = GraphqlConnection(url=config.GQL_URL)
source_connection.initialize_basic(username=config.BASIC_USERNAME, password=config.BASIC_PASSWORD)
print("XSRF token: " + source_connection.xsrf_token)

#### Zen and IAM Authentication

In [ ]:
source_connection = GraphqlConnection(url=config.GQL_URL, token_url=config.TOKEN_URL)
source_connection.initialize_oauth(client_id=config.CLIENT_ID,client_secret= config.CLIENT_SECRET, username=config.OAUTH_USERNAME, password=config.OAUTH_PASSWORD,\
                                grant_type=config.GRANT_TYPE, scope=config.SCOPE)
source_connection.get_token()
print("XSRF token: " + source_connection.xsrf_token)
print("IAM token: " + source_connection.token)
source_connection.exchange_iam_token(config.EXCHANGE_TOKEN_URL)
print("Access token: " + source_connection.token)

# Prompt for one or more classes to export

In [ ]:
selector = ClassDefinitionsSelector(config.OS_NAME, source_connection)
selector.prompt_selection()
print("Classes selected:")
for sel in selector.selections:
    jsonobj = sel.to_json()
    print(str(jsonobj))

# Initialize an Exporter object with the classes to export

In [ ]:
deplpkg = DeploymentPackage()
exporter = Exporter(source_connection, config.OS_NAME, deplpkg)
expsel = ExportSelection(selector.selections)
exporter.initialize_selection(expsel)
pkgjson = deplpkg.to_json()
exporter.export(auto_resolve_deps=True)
deplpkg.write("export.json")

print("Deployment package after objects exported:")
print(json.dumps(deplpkg.to_json(), indent=4))

# Authenticate with the target GraphQL server
(Choose one method of Authentication)


#### OAuth Authentication

In [ ]:
target_connection = GraphqlConnection(url=config.TARGET_GQL_URL, token_url=config.TARGET_TOKEN_URL)
target_connection.initialize_oauth(client_id=config.TARGET_CLIENT_ID,client_secret= config.TARGET_CLIENT_SECRET, username=config.TARGET_OAUTH_USERNAME, password=config.TARGET_OAUTH_PASSWORD,\
                                grant_type=config.TARGET_GRANT_TYPE, scope=config.TARGET_SCOPE)
target_connection.get_token()
print("XSRF token: " + target_connection.xsrf_token)
print("Authentication token: " + target_connection.token)

#### Basic Authentication

In [ ]:
target_connection = GraphqlConnection(url=config.TARGET_GQL_URL)
target_connection.initialize_basic(username=config.TARGET_BASIC_USERNAME, password=config.TARGET_BASIC_PASSWORD)
print("XSRF token: " + target_connection.xsrf_token)

#### Zen and IAM Authentication

In [ ]:
target_connection = GraphqlConnection(url=config.GQL_URL, token_url=config.TOKEN_URL)
target_connection.initialize_oauth(client_id=config.CLIENT_ID,client_secret= config.CLIENT_SECRET, username=config.OAUTH_USERNAME, password=config.OAUTH_PASSWORD,\
                                grant_type=config.GRANT_TYPE, scope=config.SCOPE)
target_connection.get_token()
print("XSRF token: " + source_connection.xsrf_token)
print("IAM token: " + source_connection.token)
source_connection.exchange_iam_token(config.EXCHANGE_TOKEN_URL)
print("Access token: " + source_connection.token)

# Load the package file previously exported and generate and run the mutations

In [ ]:
deplpkg = DeploymentPackage()
deplpkg.read('export.json')
importer = Importer(target_connection, config.TARGET_OS_NAME, deplpkg)
mutrespjsons = importer.import_package(gen_mutations=True)
print("Mutation responses:")
for mutrespjson in mutrespjsons:
    print("--------------------")
    print(json.dumps(mutrespjson, indent=4))